In [1]:
import heapq
import math
import os
import re

import aocd
import numpy as np
from IPython.display import HTML, clear_output, display
from scipy.ndimage import convolve

import time


In [2]:
p = aocd.get_puzzle(year=2024, day=15)

In [3]:
directions = {'^': (0, -1), '>': (1, 0), 'v': (0, 1), '<': (-1, 0)}

In [4]:
def get_data(test_data: bool = False):
    if test_data:
        data = p.examples[1].input_data
    else:
        data = p.input_data
    return data

In [5]:
def process_data(data):
    grid, directions = data.split("\n\n")
    grid = np.array([list(l) for l in grid.split("\n")])
    
    return grid, directions.replace("\n", "")

In [6]:
def html_grid(grid):
    colors = {
    '#': ('█', '#8d6e63'),  # Warm brown walls
    '.': ('·', '#4a4a4a'),  # Subtle grey dots
    'O': ('▣', '#f44336'),
    '@': ('◉', '#4caf50'),
    '[': ('▣', '#f44336'),
    ']': ('▣', '#f44336'),
        
}
    background = '#2d2d2d'

    cells = ""
    for row in grid:
        for c in row:
            char, color = colors.get(c, (c, '#333333'))
            cells += f'<span style="color:{color};">{char}</span>'
        cells += "<br>"
    
    html = f'''
    <div style="font-family:monospace;font-size:16px;line-height:1.1;background:{background};padding:15px;border-radius:10px;display:inline-block;">
    {cells}
    </div>
    '''
    return html
    

In [7]:
def move_loc(loc, direction):
    dx, dy = directions[direction]
    return (loc[0] + dy, loc[1] + dx)

def move_agent(loc, direction):
    dx, dy = directions[direction]
    new_loc = move_loc(loc, direction)
    
    if grid[new_loc] == '#':
        # Hit a wall, stay in place
        return loc  
    elif grid[new_loc] == '.':
        # No problen, move to new location
        grid[loc] = '.'
        grid[new_loc] = '@'
        return new_loc
    else:
        # There is a box. Move the box if possible
        box_pos = [new_loc]
        while grid[new_loc] not in ('.', '#'):
            new_loc = move_loc(new_loc, direction)
            box_pos.append(new_loc)
        #print(box_pos)
        if grid[new_loc] == '#':
            # Can't move the box, stay in place
            return loc
        elif grid[new_loc] == '.':
            # Move the box and the agent
            for i in range(len(box_pos)-1, 0, -1):
                grid[box_pos[i]] = grid[box_pos[i-1]]
            grid[loc] = '.'
            grid[box_pos[0]] = '@'
            return box_pos[0]

In [8]:
# Part 1

In [9]:
%%time
grid, moves = process_data(get_data(test_data=False))
loc = tuple(np.where(grid == "@"))

for i in moves:
    loc = move_agent(loc, i)
#    clear_output(wait=True)
#    display(HTML(html_grid(grid)))


CPU times: user 125 ms, sys: 2.85 ms, total: 127 ms
Wall time: 126 ms


In [10]:
def calculate_score(grid):
    score = 0
    for y in range(grid.shape[0]):
        for x in range(grid.shape[1]):
            if grid[y, x] == 'O':
                score += x + (y * 100)
    return score

calculate_score(grid)

1398947

# Part 2

In [11]:
def can_push_vertical(pos, dy):
    """Check if the box at pos can be pushed in direction dy."""
    row, col = pos
    cell = str(grid[row, col])
    
    # Find the full box
    if cell == '[':
        box_left, box_right = (row, col), (row, col + 1)
    elif cell == ']':
        box_left, box_right = (row, col - 1), (row, col)
    else:
        # Not a box - it's either wall or empty
        return cell == '.'
    
    # Check what's above/below both halves of the box
    next_left = str(grid[box_left[0] + dy, box_left[1]])
    next_right = str(grid[box_right[0] + dy, box_right[1]])
    
    # If either hits a wall, can't push
    if next_left == '#' or next_right == '#':
        return False
    
    # Check if both sides can move (recursively check any boxes in the way)
    left_ok = next_left == '.' or can_push_vertical((box_left[0] + dy, box_left[1]), dy)
    right_ok = next_right == '.' or can_push_vertical((box_right[0] + dy, box_right[1]), dy)
    
    return left_ok and right_ok

def do_push_vertical(pos, dy):
    """Push the box at pos in direction dy. Only call after can_push_vertical returns True."""
    row, col = pos
    cell = str(grid[row, col])
    
    if cell == '.':
        return
    
    # Find the full box
    if cell == '[':
        box_left, box_right = (row, col), (row, col + 1)
    elif cell == ']':
        box_left, box_right = (row, col - 1), (row, col)
    else:
        return
    
    # First, recursively push any boxes above/below
    next_left = str(grid[box_left[0] + dy, box_left[1]])
    next_right = str(grid[box_right[0] + dy, box_right[1]])
    
    if next_left in '[]':
        do_push_vertical((box_left[0] + dy, box_left[1]), dy)
    if next_right in '[]':
        do_push_vertical((box_right[0] + dy, box_right[1]), dy)
    
    # Then move this box
    grid[box_left[0] + dy, box_left[1]] = '['
    grid[box_right[0] + dy, box_right[1]] = ']'
    grid[box_left] = '.'
    grid[box_right] = '.'

def move_agent_p2(loc, direction):
    dx, dy = directions[direction]
    new_row, new_col = loc[0] + dy, loc[1] + dx
    new_loc = (new_row, new_col)
    
    if str(grid[new_loc]) == '#':
        return loc
    
    if str(grid[new_loc]) == '.':
        grid[loc] = '.'
        grid[new_loc] = '@'
        return new_loc
    
    if direction in '<>':
        scan_row, scan_col = new_loc
        while str(grid[scan_row, scan_col]) in '[]':
            scan_col += dx
        
        if str(grid[scan_row, scan_col]) == '#':
            return loc
        
        while scan_col != new_col:
            prev_col = scan_col - dx
            grid[scan_row, scan_col] = grid[scan_row, prev_col]
            scan_col = prev_col
        
        grid[loc] = '.'
        grid[new_loc] = '@'
        return new_loc
    
    else:
        if can_push_vertical(new_loc, dy):
            do_push_vertical(new_loc, dy)
            grid[loc] = '.'
            grid[new_loc] = '@'
            return new_loc
        return loc

In [12]:
def expand_grid(original_grid):
    """Transform part 1 grid to part 2 format."""
    expansion = {'#': '##', 'O': '[]', '.': '..', '@': '@.'}
    rows, cols = original_grid.shape
    new_grid = np.empty((rows, cols * 2), dtype='<U1')
    
    for row in range(rows):
        for col in range(cols):
            cell = original_grid[row, col]
            expanded = expansion[cell]
            new_grid[row, col * 2] = expanded[0]
            new_grid[row, col * 2 + 1] = expanded[1]
    
    return new_grid
    

In [13]:
%%time
grid, moves = process_data(get_data(test_data=False))
grid = expand_grid(grid)

loc = tuple(np.argwhere(grid == '@')[0])

for i in moves:
    loc = move_agent_p2(loc, i)
#    clear_output(wait=True)
#    display(HTML(html_grid(grid)))


CPU times: user 27.8 ms, sys: 1.03 ms, total: 28.8 ms
Wall time: 28.2 ms


In [14]:
def calculate_score(grid):
    score = 0
    for y in range(grid.shape[0]):
        for x in range(grid.shape[1]):
            if grid[y, x] == '[':
                score += x + (y * 100)
    return score

calculate_score(grid)

1397393

In [15]:
clear_output(wait=True)
display(HTML(html_grid(grid)))